In [1]:
!pip install pybit


In [2]:
from pybit.unified_trading import HTTP
session = HTTP(testnet=True)
print(session.get_kline(
    category="inverse",
    symbol="BTCUSD",
    interval=60,
    start=1670601600000,
    end=1670608800000,
))

{'retCode': 0, 'retMsg': 'OK', 'result': {'symbol': 'BTCUSD', 'category': 'inverse', 'list': [['1670608800000', '17071', '17073', '17027', '17055.5', '268611', '15.74462667'], ['1670605200000', '17071.5', '17071.5', '17061', '17071', '4177', '0.24469757'], ['1670601600000', '17086.5', '17088', '16978', '17071.5', '6356', '0.37288112']]}, 'retExtInfo': {}, 'time': 1729700686529}


In [8]:
from pybit.unified_trading import HTTP
import pandas as pd

# Khởi tạo phiên làm việc với testnet
session = HTTP(testnet=True)

# Định nghĩa các tham số để lấy dữ liệu
category = "inverse"
symbol = "BTCUSD"
interval = 60  # khoảng thời gian 60 phút
start = 1670601600000  # thời gian bắt đầu (milliseconds)
end = 1670608800000    # thời gian kết thúc (milliseconds)

# Lấy dữ liệu nến
response = session.get_kline(
    category=category,
    symbol=symbol,
    interval=interval,
    start=start,
    end=end,
)

# Kiểm tra phản hồi và xử lý dữ liệu
if response['result']:
    # Chuyển đổi dữ liệu thành DataFrame
    data = response['result']
    df = pd.DataFrame(data)




In [49]:
from pybit.unified_trading import HTTP
import pandas as pd
from datetime import datetime

# Khởi tạo phiên làm việc với testnet
session = HTTP(testnet=True)

# Định nghĩa các tham số để lấy dữ liệu
category = "inverse"
symbol = "BTCUSD"
interval = 60  #khoảng thời gian 60 phút


# Đặt thời gian bắt đầu và kết thúc
start_date = "2024-01-01"
end_date = "2024-06-1"  # Để bao gồm hết tháng 6

# Chuyển đổi thời gian sang định dạng milliseconds
start_timestamp = int(pd.Timestamp(start_date).timestamp() * 1000)  # milliseconds
end_timestamp = int(pd.Timestamp(end_date).timestamp() * 1000)  # milliseconds

start = start_timestamp  # thời gian bắt đầu (milliseconds)
end = start_timestamp + 604800000    # thời gian kết thúc (milliseconds)



# Lấy dữ liệu nến
response = session.get_kline(
    
    category=category,
    symbol=symbol,
    interval=interval,
    start=start,
    end=end,
)

# Kiểm tra phản hồi và xử lý dữ liệu
if response['retCode'] == 0 and 'list' in response['result']:
    # Lấy danh sách nến
    candles = response['result']['list']
    
    # Chuyển đổi dữ liệu thành DataFrame
    df = pd.DataFrame(candles, columns=['Time (UTC+3)', 'Open Price', 'Highest Price', 'Lowest Price', 'Close Price', 'Trade Volume', 'Another Value'])

    # Chuyển đổi cột thời gian từ milliseconds sang định dạng datetime
    df['Time (UTC+3)'] = pd.to_datetime(df['Time (UTC+3)'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Etc/GMT-3')

    # Chọn cột cần thiết
    df = df[['Time (UTC+3)', 'Open Price', 'Highest Price', 'Lowest Price', 'Close Price', 'Trade Volume']]
    
    # Đổi kiểu dữ liệu cho các cột số
    df[['Open Price', 'Highest Price', 'Lowest Price', 'Close Price', 'Trade Volume']] = df[['Open Price', 'Highest Price', 'Lowest Price', 'Close Price', 'Trade Volume']].astype(float)

    # Hiển thị DataFrame
    print(df)
else:
    print("Không có dữ liệu nến hoặc có lỗi trong phản hồi.")


                 Time (UTC+3)  Open Price  Highest Price  Lowest Price  \
0   2024-01-08 03:00:00+03:00     43946.0        44003.0       43656.5   
1   2024-01-08 02:00:00+03:00     43893.5        44046.0       43636.5   
2   2024-01-08 01:00:00+03:00     44259.5        44259.5       43813.0   
3   2024-01-08 00:00:00+03:00     44147.5        44272.0       44146.5   
4   2024-01-07 23:00:00+03:00     44262.0        44292.0       44147.5   
..                        ...         ...            ...           ...   
164 2024-01-01 07:00:00+03:00     42361.5        42427.5       42251.0   
165 2024-01-01 06:00:00+03:00     42606.5        42617.0       42281.0   
166 2024-01-01 05:00:00+03:00     42639.5        42657.0       42533.0   
167 2024-01-01 04:00:00+03:00     42503.5        43500.0       42452.5   
168 2024-01-01 03:00:00+03:00     42301.5        42566.5       42288.0   

     Close Price  Trade Volume  
0        43656.5    41759487.0  
1        43946.0    56790177.0  
2        438

C:\Users\tranm\AppData\Local\Temp\ipykernel_27956\2061953119.py:46: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['Time (UTC+3)'] = pd.to_datetime(df['Time (UTC+3)'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Etc/GMT-3')
